In [1]:
import pandas as pd
import numpy as np

import keras
import keras.backend as K
from keras import layers, models, optimizers
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import Huber

from sklearn.preprocessing import LabelEncoder

import gc
import os



Using TensorFlow backend.


In [ ]:
keras.wrappers.scikit_learn.K

In [2]:
tr = pd.read_csv('./data/train.csv')
te = pd.read_csv('./data/test.csv')

sub = pd.read_csv('./data/sample_submission.csv', index_col=0)

In [3]:
# target_lbe = LabelEncoder().fit(target)

# t = target_lbe.transform(target)

column_number = {}
for i, column in enumerate(sub.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

tr['type_num'] = tr['type'].apply(lambda x: to_number(x, column_number))



In [4]:
target = tr['type_num']
t = target.copy()

train_X = tr.drop(['id', 'type', 'type_num'], axis=1)
test_X = te.drop(['id',], axis=1)



In [5]:
t

0          8
1          8
2          8
3          8
4         10
          ..
199986     8
199987     6
199988    10
199989     6
199990     8
Name: type_num, Length: 199991, dtype: int64

In [6]:
train_X['index'] = np.ones(len(train_X))
test_X['index'] = np.ones(len(test_X))*2

merge = pd.concat([train_X, test_X], ignore_index=True)

k = train_X.columns[1:-1]

In [7]:
merge = pd.concat([merge, pd.get_dummies(merge['fiberID'], prefix='fiberID')], axis=1)
merge = merge.drop('fiberID', axis=1)


In [8]:
train_X = merge[merge['index'] == 1]
train_X = train_X.drop('index', axis=1)
test_X = merge[merge['index'] == 2]
test_X = test_X.drop('index', axis=1)
test_X.index = range(len(test_X))

In [9]:
tr_X = train_X[k]
te_X = test_X[k]

tr_X = (tr_X - np.mean(tr_X))/np.std(tr_X)
te_X = (te_X - np.mean(te_X))/np.std(te_X)

train_X[k] = tr_X
test_X[k] = te_X

In [10]:
es = EarlyStopping(patience=20, restore_best_weights=True)
def schedule(epoch):
    if epoch < 80:
        return 0.0005
    elif epoch < 160:
        return 0.0001
    else:
        return 0.00001
lrs = LearningRateScheduler(schedule)

In [11]:
def DenseBlock(n, activation='relu'):
    def f(x):
        x = layers.Dense(n, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.001))(x)
#         x = layers.BatchNormalization()(x)
        x = layers.advanced_activations.LeakyReLU(0.3)(x)
        
        return x
    return f


In [21]:
K.clear_session()

inputs = layers.Input(shape=(train_X.shape[1], ))

x1 = layers.Dense(256*4, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(inputs)
x1 = layers.BatchNormalization()(x1)
x1 = layers.Activation('relu')(x1)

x1 = DenseBlock(256*4)(x1)
# x1 = DenseBlock(256*3)(x1)
x1 = DenseBlock(256*2)(x1)
x1 = DenseBlock(256)(x1)
x1 = DenseBlock(64)(x1)
# x1 = DenseBlock(32)(x1)

x2 = layers.Dense(256*4, kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0001))(inputs)
x2 = layers.BatchNormalization()(x2)
x2 = layers.Activation('relu')(x2)

x2 = DenseBlock(256, 'tanh')(x2)
# x2 = DenseBlock(256, 'tanh')(x2)
x2 = DenseBlock(128, 'tanh')(x2)
x2 = DenseBlock(64, 'tanh')(x2)
# x2 = DenseBlock(32, 'tanh')(x2)


x = layers.concatenate([x1, x2])

x = layers.Dense(32, activation='relu')(x)

x = layers.Dense(19, activation='softmax')(x1)

m = models.Model(inputs, x)

# sparse_categorical_crossentropy
m.compile(optimizer = 'adam',
         loss = 'sparse_categorical_crossentropy', 
          metrics = ['acc'])

In [ ]:
m.fit(train_X, t, 
     epochs = 1000,
     validation_split=0.15,
     batch_size=1024*16,
      callbacks=[es, lrs]
     )

In [ ]:
m.evaluate(train_X, t)

In [ ]:
m.evaluate(train_X, t)

In [ ]:
y_pred = m.predict(test_X)
submission = pd.DataFrame(data=y_pred, columns=sub.columns, index=sub.index)
submission.to_csv('./sub/nn6.csv', index=True)


In [ ]:
kk = pd.DataFrame(m.predict(test_X), columns=target_lbe.classes_)
sub[sub.columns[1:]] = kk[sub.columns[1:]]

In [ ]:
sub.head()

In [ ]:
sub.to_csv('./sub/nn4.csv', index=False)

In [ ]:
print(train_X.shape, test_X.shape)